In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report



In [ ]:
data=pd.read_csv("/content/fraudTrain.csv");
data

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810,7810,2019-01-06 01:59:19,6524376634971418,fraud_Murray-Smitham,grocery_pos,132.27,Jasmine,Foster,F,954 White Alley Apt. 251,...,34.3470,-87.7154,5778.0,"Engineer, aeronautical",1952-12-05,e11d567d20b74f412c644122f57b7152,1.325815e+09,34.011938,-88.080823,0.0
7811,7811,2019-01-06 01:59:25,2348245054386329,fraud_Hackett-Lueilwitz,grocery_pos,83.51,Justin,Gay,M,268 Hayes Rue Suite 811,...,42.1767,-79.9416,2518.0,Event organiser,1946-02-02,dc5380f7b501f8fae23741ae9426ea87,1.325815e+09,41.328228,-79.226936,0.0
7812,7812,2019-01-06 02:00:33,2283743876903625,fraud_Koepp-Witting,grocery_pos,64.04,Shannon,Williams,F,9345 Spencer Junctions Suite 183,...,34.0770,-84.3033,165556.0,Prison officer,1997-12-27,eefe1322ac9155ed949e3c71b35ccc37,1.325815e+09,33.376781,-83.592263,0.0
7813,7813,2019-01-06 02:01:43,213195754007681,"fraud_Robel, Cummerata and Prosacco",gas_transport,66.86,Zachary,Allen,M,69252 Oconnor Alley Apt. 153,...,41.2001,-92.1354,568.0,Commercial horticulturist,1969-07-24,bb4b4d3d673dddf931fc71c12e090b1d,1.325815e+09,40.955293,-92.395676,0.0


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['unix_time']=le.fit_transform(data['unix_time'])
data['merch_lat']=le.fit_transform(data['merch_lat'])
data['merch_long']=le.fit_transform(data['merch_long'])
data['is_fraud']=le.fit_transform(data['is_fraud'])
data['merchant']=le.fit_transform(data['merchant'])
data['category']=le.fit_transform(data['category'])
data['first']=le.fit_transform(data['first'])
data['last']=le.fit_transform(data['last'])
data['street']=le.fit_transform(data['street'])
data['job']=le.fit_transform(data['job'])
data['trans_date_trans_time']=le.fit_transform(data['trans_date_trans_time'])
data['cc_num']=le.fit_transform(data['cc_num'])
data['trans_num']=le.fit_transform(data['trans_num'])


In [ ]:
data

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,0,409,514,8,4.97,150,17,F,518,...,36.0788,-81.1781,3495.0,354,1988-03-09,349,0,2356,5278,0
1,1,1,36,241,4,107.23,289,153,F,402,...,48.8878,-118.2105,149.0,409,1978-06-21,924,1,7755,521,0
2,2,2,218,390,0,220.11,104,367,M,551,...,42.1808,-112.2620,4154.0,293,1962-01-19,4892,2,6535,779,0
3,3,3,467,360,2,45.00,151,442,M,848,...,46.2306,-112.1138,1939.0,314,1967-01-12,3234,3,7552,755,0
4,4,4,340,297,9,41.96,315,145,M,387,...,38.4207,-79.4629,99.0,110,1986-03-28,4976,4,3410,6277,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810,7810,7729,789,447,4,132.27,144,134,F,861,...,34.3470,-87.7154,5778.0,146,1952-12-05,6904,7729,1581,3851,0
7811,7811,7730,398,217,4,83.51,175,149,M,246,...,42.1767,-79.9416,2518.0,177,1946-02-02,6757,7730,5424,6119,0
7812,7812,7731,387,329,4,64.04,278,446,F,840,...,34.0770,-84.3033,165556.0,336,1997-12-27,7303,7731,1324,4821,0
7813,7813,7732,280,518,2,66.86,330,4,M,631,...,41.2001,-92.1354,568.0,88,1969-07-24,5709,7732,5129,2927,0


In [ ]:
x = x.drop('trans_num', axis=1)

In [ ]:
x=data.iloc[:,[1,2,18]]
y=data.iloc[:,-1]

In [ ]:
x

,trans_date_trans_time,cc_num,trans_num
0,0,409,349
1,1,36,924
2,2,218,4892
3,3,467,3234
4,4,340,4976
...,...,...,...
7810,7729,789,6904
7811,7730,398,6757
7812,7731,387,7303
7813,7732,280,5709


In [ ]:
y

0       0
1       0
2       0
3       0
4       0
       ..
7810    0
7811    0
7812    0
7813    0
7814    2
Name: is_fraud, Length: 7815, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
Se=StandardScaler()
x_train=Se.fit_transform(x_train)
x_test=Se.transform(x_test)

In [ ]:
rf_classifier=RandomForestClassifier(n_estimators=100,random_state=42)

In [ ]:

rf_classifier.fit(x_train, y_train) # Fit the model to the training data
y_pred=rf_classifier.predict(x_test) # Predict the target variable for the test data

In [ ]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(6252, 3) (1563, 3) (6252,) (1563,)


In [ ]:
accuracy=accuracy_score(y_test,y_pred)
classification_rep=classification_report(y_test,y_pred)

In [ ]:
print("accuracy:,accuracy")
print("classification report:\n",classification_rep)

accuracy:,accuracy
classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1554
           1       1.00      0.22      0.36         9

    accuracy                           1.00      1563
   macro avg       1.00      0.61      0.68      1563
weighted avg       1.00      1.00      0.99      1563

